In [1]:
from utils.t5 import *

In [2]:
input_data_name = "claim_LOF_base_0.11_data_explanation_prep_4.pickle"                            #"LOF_base_0.45_0.53_removed_inlier_outlier_23.782_full.pickle"     # "LOF_base_0.46_0.54_removed_inlier_outlier_0.51_full.pickle"
data_inpit_dir = "./Data/Selection/"                              #"./Data/Selection/" "./Data/Preprocessed/"   
output_dir = "./Data/Models/"
source_column = "source_text"    #source_text_shorter           " statement_explanation_prep"  #"explanation_prep"  "statement_explanation_prep"                     #"source_text_shorter"  # "source_text_shorter" source_text
target_column =  "target_text"              #"shortExplanation_prep"                         #"target_text"

no_workers = 1

imput_data_path = data_inpit_dir + input_data_name
new_model_name = "d-t5-{}_{}".format(source_column, input_data_name)

In [5]:
torch.cuda.get_device_name(0)

'A100-SXM4-40GB'

In [6]:
new_model_name

'd-t5-source_text_claim_LOF_base_0.11_data_explanation_prep_4.pickle'

In [4]:
data = pd.read_pickle(imput_data_path)

In [5]:
train, dev_test = train_test_split(data, test_size = 0.2, random_state = 42)
dev, test = train_test_split(dev_test, test_size = 0.5, random_state = 42)

In [6]:
train['target_text'] = train[target_column]
train['source_text'] = "summarize: " + train[source_column]

train['target_len'] = train["target_text"].str.split().str.len()
train['source_len'] = train["source_text"].str.split().str.len()
train[['target_len','source_len']].describe()

,target_len,source_len
count,10312.000000,10312.000000
mean,85.508146,794.291020
std,41.681467,288.230128
min,2.000000,41.000000
25%,58.000000,589.000000
50%,80.000000,757.000000
75%,106.250000,954.000000
max,775.000000,2936.000000


In [8]:
sum(train.source_len.to_list())

8190729

In [17]:
len(train[train.target_len>150])

590

In [18]:
len(train[train.source_len>1200])

897

In [10]:
len(train)

10312

In [9]:
dev['target_text'] = dev[target_column]
dev['source_text'] = "summarize: " + dev[source_column]

dev['target_len'] = dev["target_text"].str.split().str.len()
dev['source_len'] = dev["source_text"].str.split().str.len()
dev[['target_len','source_len']].describe()

,target_len,source_len
count,1289.000000,1289.000000
mean,85.415050,803.556245
std,41.921667,303.170137
min,9.000000,207.000000
25%,58.000000,592.000000
50%,80.000000,758.000000
75%,106.000000,963.000000
max,486.000000,2922.000000


In [11]:
sum(dev.source_len.to_list())

1035784

In [12]:
test['target_text'] = test[target_column]
test['source_text'] = "summarize: " + test[source_column]

test['target_len'] = test["target_text"].str.split().str.len()
test['source_len'] = test["source_text"].str.split().str.len()
test[['target_len','source_len']].describe()

,target_len,source_len
count,1290.000000,1290.000000
mean,85.591473,789.255039
std,48.017999,289.329616
min,4.000000,94.000000
25%,59.000000,592.250000
50%,81.000000,746.000000
75%,106.000000,939.750000
max,1121.000000,2736.000000


In [13]:
sum(test.source_len.to_list())

1018139

In [12]:
train = train[["target_text", "source_text"]]
dev = dev[["target_text","source_text"]]

In [13]:
model = SimpleT5()

In [14]:
model.from_pretrained(model_type="t5",model_name = "t5-base") # large  "google/mt5-base"

In [15]:
import gc
#del data, model
gc.collect()
import torch
torch.cuda.empty_cache()

In [16]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [17]:
model.train(train_df = train, #LOF + Bert 11
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-69415620-de6c-329d-ef8d-9c9c937f4850]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.308


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9884


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7726


In [17]:
model.train(train_df = train, #claim + LOF + Bert 11
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-69415620-de6c-329d-ef8d-9c9c937f4850]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2405


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9311


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7228


In [17]:
model.train(train_df = train, #claim + LOF + Bert tunned 11
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-69415620-de6c-329d-ef8d-9c9c937f4850]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2426


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.935


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7229


In [17]:
model.train(train_df = train, #LOF + Bert tunned 13
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3119


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9923


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7793


In [17]:
model.train(train_df = train, #LOF + Bert 13
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3403


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0105


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7822


In [17]:
model.train(train_df = train, #claim + LOF + Bert 13
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2374


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9311


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7217


In [17]:
model.train(train_df = train, #claim + LOF + Bert tunned 13
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2465


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9367


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7295


In [17]:
model.train(train_df = train, #claim + LOF + Bert tunned 15
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2632


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9445


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7419


In [17]:
model.train(train_df = train, #claim + LOF + Bert
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2418


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9356


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7266


In [18]:
model.train(train_df = train, # LOF + Bert tunned
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3681


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0201


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8071


In [19]:
model.train(train_df = train, # LOF + Bert
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3142


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0009


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7837


In [17]:
model.train(train_df = train, # claim base
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-b1be761c-039f-c170-b0c1-c82ee601e195]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.228


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9314


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7178


In [17]:
model.train(train_df = train, #base 
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-cd2a3fdb-d296-06a1-ff6a-5b6d54df0acc]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2812


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9751


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7619


In [17]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-69415620-de6c-329d-ef8d-9c9c937f4850]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2256


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9291


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7165


In [17]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-1cad794e-e425-8f4b-3a02-a512bba3497a]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/storage/brno3-cerit/home/vajp02/.conda/envs/myenv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/storage/brno3-cerit/home/vajp02/.conda/envs/myenv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2921


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9845


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7736


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-bceb1af3-48fa-7240-a710-a81ba0d3ebc5]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2872


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9705


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7516


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2267


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.922


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7125


In [18]:
model.train(train_df = train, #45_57
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2506


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9477


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.736


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-c0eb6ae4-7a40-b947-996f-5811490f6f65]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.235


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9299


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7274


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2565


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9485


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7415


In [18]:
model.train(train_df = train, #45_55
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2759


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9622


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7635


In [36]:
model.train(train_df = train, #45_54
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3017


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.992


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7793


In [18]:
model.train(train_df = train, # 57
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2577


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9523


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7442


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3315


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0217


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8072


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3992


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0866


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8762


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3697


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0543


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8423


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-c85b1185-eb3f-7f99-412a-2486fe49bf7c]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3461


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0335


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8241


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-c85b1185-eb3f-7f99-412a-2486fe49bf7c]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2937


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9792


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7665


In [17]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 100,  #100 max_shortexplanation_tokens
            batch_size = 9, max_epochs = 3, # 18
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-bceb1af3-48fa-7240-a710-a81ba0d3ebc5]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Metric val_loss improved by 0.091 >= min_delta = 0.0. New best score: 1.882


Average training loss for epoch 1 equal to 1.7684


Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 1 records. Best score: 1.882. Signaling Trainer to stop.


Average training loss for epoch 2 equal to 1.5647


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 100,  #100 max_shortexplanation_tokens
            batch_size = 9, max_epochs = 6, # 18
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-a527da35-0f6b-a3bf-de28-5bb4f9ba4001]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Average training loss for epoch 3 equal to 1.444


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 4 equal to 1.2971


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 5 equal to 1.1629
